# ⚡ MOSFET 101

No introduction to analog IC design would be complete without a whistlestop tour of the basics of MOSFETs.

🎥 Video Insight:

<iframe width="560" height="315" src="https://www.youtube.com/embed/xeEQdo5MOGo?si=hZO2j8_oS0RD1krW" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

<iframe width="560" height="315" src="https://www.youtube.com/embed/pwrrcRuxjlQ?si=sWzgzv6njVdmNq2r" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

📝 Large-signal MOSFET behavior is summarized in the table below:

| Condition/Parameter          | Equation or Description                                                                 |
|------------------------------|----------------------------------------------------------------------------------------|
| **1-D MOSFET Model**         |                                                                                        |
| Drain Current ($I_D$)        | $I_D = \frac{W}{L} \mu C_{ox} \left( V_{GS} - V_{th} \right) V_{DS}$ (Triode Region) |
| **MOSFET in Saturation**     |                                                                                        |
| Drain Current ($I_D$)        | $I_D = \frac{1}{2} \frac{W}{L} \mu C_{ox} \left( V_{GS} - V_{th} \right)^2$           |
| **Channel Length Modulation**|                                                                                        |
| Drain Current ($I_D$)        | $I_D = \frac{1}{2} \frac{W}{L} \mu C_{ox} \left( V_{GS} - V_{th} \right)^2 (1 + \lambda V_{DS})$ |
| **The Body Effect**          |                                                                                        |
| Body Effect Parameter ($\gamma$) | $\gamma = \frac{\sqrt{2q \epsilon_{Si} N_{sub}}}{C_{ox}}$                           |
| Threshold Voltage ($V_{th}$) | $V_{th} = V_{th0} + \gamma \left( \sqrt{2\phi_f + V_{SB}} - \sqrt{2\phi_f} \right)$   |

Where:
- $W$ and $L$ are the width and length of the MOSFET, respectively.
- $\mu$ is the carrier mobility.
- $C_{ox}$ is the oxide capacitance per unit area.
- $V_{GS}$ is the gate-source voltage.
- $V_{DS}$ is the drain-source voltage.
- $V_{th0}$ is the threshold voltage without the body effect.
- $\gamma$ is the body effect parameter.
- $2\phi_f$ is twice the Fermi potential.
- $V_{SB}$ is the source-body voltage.
- $\lambda$ is the channel length modulation parameter.
- $N_{sub}$ is the substrate doping concentration.
- $\epsilon_{Si}$ is the permittivity of silicon.
- $q$ is the electron charge.

Note: These equations are basic and idealized. In real-world applications, more detailed and complex models might be needed to accurately describe the behavior of MOSFETs.

💡 To experiment with, we'll first cover the large-signal behavior of a standard 1.8V NMOS from the SKY130 PDK:

##### 🧠 Exercises

1. **Exploration**: See if you can identify the triode, saturation and short-channel regions of MOSFET operation

2. **Manipulation**: Change the `w` and `l` parameters of the device, do you get the changes you'd expect?

In [ ]:
# Import the hdl21 library
import hdl21 as h
from hdl21.prefix import u
import vlsirtools.spice as vsp

# Import the SKY130 PDK
from sitepdks import *
import sky130_hdl21 as s
from sky130_hdl21 import primitives as p

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# Don't touch any of this!
def get_sim(device : h.Module,vgs=1.8) -> h.sim.Sim:

    @h.sim.sim
    class DeviceSim:

        @h.module
        class Tb:

            # Set up our ports
            VSS = h.Port()
            VDD = h.Signal()
            
            # Define our voltages
            vdd = h.DcVoltageSource(dc=vgs)(p=VDD, n=VSS)
            sweep = h.PulseVoltageSource(delay=0,
                                         v1=0,
                                         v2=1.8,
                                         period=1,
                                         rise=1,
                                         fall=1,
                                         width=1)(n=VSS)
            # Instantiate our device
            DUT = device(d=sweep.p, g=VDD, s=VSS, b=VSS)

        tran = h.sim.Tran(tstop=1,tstep=0.001)
        inc = s.install.include(h.pdk.Corner.TYP)

    return DeviceSim

opts = vsp.SimOptions(
            simulator=vsp.SupportedSimulators.NGSPICE,
            fmt=vsp.ResultFormat.SIM_DATA,
            rundir="./scratch",
        )

import numpy as np
import matplotlib.pyplot as plt

# Modify your device here!
device2 = p.NMOS_1p8V_STD(w=0.65, l=0.15)

# Modify vgs and simulate
sim = get_sim(device=device2,vgs=1.8)
rvs = sim.run(opts=opts)

# Plot the results
x = rvs.an[0].data['v(xtop.sweep_p)']
y = -rvs.an[0].data['i(v.xtop.vvdd)']*1000

plt.plot(x,y)

### Small-Signal Models

MOSFETs are complicated non-linear devices, and often-times, we only require a small signal approximation of their behavior is tease out the intuitive properties that we need to think about design

📝 The attributes of small-signal models are summarized in the table below :

| Model/Parameter                 | Low-Frequency (Hybrid-π)                 | High-Frequency                           |
|---------------------------------|------------------------------------------|------------------------------------------|
| **π-Model**                     |                                          |                                          |
| Input Resistance ($r_{gs}$)   | Infinite (MOSFET is voltage-controlled)  | Infinite                                 |
| Transconductance ($g_m$)      | $g_m = \frac{\partial I_D}{\partial V_{GS}}$ | $g_m = \frac{\partial I_D}{\partial V_{GS}}$ |
| Output Resistance ($r_{ds}$)  | $\frac{1}{\lambda I_D}$ (if channel length modulation is considered) | $\frac{1}{\lambda I_D + g_{mb}V_{BS}}$  |
| Gate-Source Capacitance ($C_{gs}$) | Not considered                      | Included                                 |
| Gate-Drain Capacitance ($C_{gd}$)  | Not considered                      | Included                                 |
| **T-Model**                     |                                          |                                          |
| Input Resistance ($r_{gs}$)   | Infinite                                 | Infinite                                 |
| Transresistance ($r_m$)       | $r_m = \frac{1}{g_m}$                   | $r_m = \frac{1}{g_m}$                   |
| Output Resistance ($r_{ds}$)  | $\frac{1}{\lambda I_D}$ (if channel length modulation is considered) | $\frac{1}{\lambda I_D + g_{mb}V_{BS}}$  |
| Gate-Source Capacitance ($C_{gs}$) | Not considered                      | Included                                 |
| Gate-Drain Capacitance ($C_{gd}$)  | Not considered                      | Included                                 |

Notes:
- The π-model is often used for its simplicity and is more intuitive for many circuit analyses.
- The T-model, while less intuitive, can sometimes simplify certain analyses, especially when dealing with source degeneration.
- In the high-frequency model, the parasitic capacitances ($C_{gs}$ and $C_{gd}$) become significant and can affect the frequency response of the MOSFET.
- When operating in triode, the model is assumed to include no voltage controlled current sources and is instead assumed to be totally passive
- $g_{mb}$ is the transconductance due to the body effect.
- $V_{BS}$ is the body-source voltage.
- $\lambda$ is the channel length modulation parameter.

💡 In the next few simulations, we investigate the small-signal approximation of MOSFETs, how well and under what conditions they can emulate fully-simulated devices:

##### 🧠 Exercises

